In [1]:
from propensity_prediction.tasks.LTV_prediction.main import LTV_prediction

In [2]:
global_config = {
    'tasks':'LTV_Prediction', 
    'data_config':{
        "path": "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx",
        "History":{
            "user_id": "CustomerID",
            "invoice_date": "InvoiceDate",
            "invoice_no": "InvoiceNo",
            "num_units": "Quantity",
            "unit_price": "UnitPrice",
            "metadata": {}},
        'Constraint': {
            'event_constraint': 0.5 # Number or tr months model will use to train 
        }
}}
global_config

{'tasks': 'LTV_Prediction',
 'data_config': {'path': 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx',
  'History': {'user_id': 'CustomerID',
   'invoice_date': 'InvoiceDate',
   'invoice_no': 'InvoiceNo',
   'num_units': 'Quantity',
   'unit_price': 'UnitPrice',
   'metadata': {}},
  'Constraint': {'event_constraint': 0.5}}}

In [3]:
predict, evaluate, ens_model = LTV_prediction(global_config)

Prepare contextual data
Prepare data for models
Train ensemble LTVPrediction
+ Prepare model LTVPrediction with label CLV
    Training with minibacth  0
	+ [iteration 0001] loss: 6873.0605
	+ [iteration 0100] loss: 6780.2559
	+ [iteration 0200] loss: 6694.8779
	+ [iteration 0300] loss: 6617.4663
	+ [iteration 0400] loss: 6547.5156
	+ [iteration 0500] loss: 6484.5308
    Training with minibacth  1
	+ [iteration 0001] loss: 2411.5835
	+ [iteration 0100] loss: 1619.6514


/root/.local/share/virtualenvs/ds-CSRLTXzN/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


	+ [iteration 0200] loss: 1195.7729
	+ [iteration 0300] loss: 917.1004
	+ [iteration 0400] loss: 718.4672
	+ [iteration 0500] loss: 572.5695
    Training with minibacth  2
	+ [iteration 0001] loss: 185.2623
	+ [iteration 0100] loss: 180.6320
	+ [iteration 0200] loss: 177.3831
	+ [iteration 0300] loss: 174.1662
	+ [iteration 0400] loss: 171.0064
	+ [iteration 0500] loss: 167.9263
    Training with minibacth  3
	+ [iteration 0001] loss: 246.0158
	+ [iteration 0100] loss: 218.6719
	+ [iteration 0200] loss: 198.6565
	+ [iteration 0300] loss: 184.6376
	+ [iteration 0400] loss: 174.0488
	+ [iteration 0500] loss: 165.5104
    Training with minibacth  4
	+ [iteration 0001] loss: 340.9437
	+ [iteration 0100] loss: 335.4499
	+ [iteration 0200] loss: 330.2024
	+ [iteration 0300] loss: 325.2564
	+ [iteration 0400] loss: 320.6134
	+ [iteration 0500] loss: 316.2700
----Running time:  0.8326113224029541


In [4]:
print (predict['model_name'])
predict['predict']

LTVPrediction


month_yr,CustomerID,predict
0,12346.0,0.000000
1,12347.0,3020.774414
2,12348.0,406.447998
3,12349.0,2336.933594
4,12350.0,0.000000
...,...,...
4367,18280.0,0.000000
4368,18281.0,0.000000
4369,18282.0,277.226624
4370,18283.0,1708.450317


# Test functions

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


from propensity_prediction.auto_preprocess.base import Base_Preprocess

from propensity_prediction.config.model_config import LTVPrediction_ModelConfig
from propensity_prediction.config.data_config import LTVPrediction_DataConfig
from propensity_prediction.config.base import Base_TaskConfig
from propensity_prediction.tasks.LTV_prediction.LTV_prediction import LTVPrediction_Task

class LTVPrediction_Config(Base_TaskConfig):
	def __init__(self, global_config):
		data_config = LTVPrediction_DataConfig(global_config['data_config'])
		model_config = LTVPrediction_ModelConfig()
		super().__init__(data_config, model_config)

def prepare_tasks(global_config):
	task_config = LTVPrediction_Config(global_config)
	data_path = task_config.get_datapath()
	key_types, feature_types = task_config.parse_data_config()
	df = pd.read_excel(data_path, nrows = 1000)

	auto_preprocess_obj = Base_Preprocess(key_types, feature_types)
	data_df, preprocess_feature_types = auto_preprocess_obj.auto_preprocess(df)

	# date_column_name = task_config.data_config.get_column_name(key='invoice_date') 
	# data_train, data_test = data_df[data_df[date_column_name]<''], data_df[data_df[date_column_name]>=''] 

	data_train, data_test = train_test_split(data_df, test_size = 0.2, random_state = 0)
	task_config.update_metadata(preprocess_feature_types)
	return data_train, data_test, task_config

In [6]:
data_train, data_test, task_config = prepare_tasks(global_config)

In [7]:
ens_model.get_probabilities(data_train, predicting_batchsize = 100)

In [11]:
ens_model.predict(data_test, predicting_batchsize = 100)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (37x1 and 6x1)